In [1]:
# https://pypi.tuna.tsinghua.edu.cn/simple
import anndata as ad
import scanpy as sc
import squidpy as sq
import cellcharter as cc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


In [2]:
rnadata = sc.read_h5ad("../20240502-SPACseq/RNA/Multitime.clustered.h5")

In [3]:
import scvi
model = scvi.model.SCVI.load("/home/wpy/stereoseq/20240502-SPACseq/scripts/scvi.model")

Global seed set to 0


INFO     File /home/wpy/stereoseq/20240502-SPACseq/scripts/scvi.model/model.pt already downloaded                  


/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/anndata/__init__.py:55: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(
/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


In [4]:
model

SCVI Model with the following params: 
n_hidden: 128, n_latent: 20, n_layers: 10, dropout_rate: 0.1, dispersion: gene, gene_likelihood: poisson, 
latent_distribution: normal
Training status: Trained
Model's adata is minified?: False

In [5]:
rnadata.obsm["X_scVI"] = model.get_latent_representation(rnadata).astype(np.float32)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(
/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/abc.py:119: FutureWarning: SparseDataset is deprecated and will be removed in late 2024. It has been replaced by the public classes CSRDataset and CSCDataset.

For instance checks, use `isinstance(X, (anndata.experimental.CSRDataset, anndata.experimental.CSCDataset))` instead.

For creation, use `anndata.experimental.sparse_dataset(X)` instead.

  return _abc_instancecheck(cls, instance)


In [6]:
rnadata.obs["marker"] = rnadata.obs["marker"].astype("category")

In [7]:
sq.gr.spatial_neighbors(rnadata, library_key="marker", coord_type="generic", delaunay=False, spatial_key="spatial", radius=150)

In [8]:
cc.gr.remove_long_links(rnadata)

In [9]:
cc.gr.aggregate_neighbors(rnadata, n_layers=3, use_rep="X_scVI", out_key="X_cellcharter", sample_key="marker")

100%|██████████| 6/6 [00:02<00:00,  2.46it/s]


In [10]:
autok = cc.tl.ClusterAutoK(
    n_clusters = (5,15),
    max_runs = 5,
    model_params = dict(
        random_state = 114514
    )
)

In [11]:
autok.fit(rnadata, use_rep="X_cellcharter")

Iteration 1/5


  0%|          | 0/13 [00:00<?, ?it/s]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
  8%|▊         | 1/13 [00:13<02:41, 13.49s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
 15%|█▌        | 2/13 [00:29<02:45, 15.05s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabri

Iteration 2/5


  0%|          | 0/13 [00:00<?, ?it/s]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
  8%|▊         | 1/13 [00:12<02:29, 12.47s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
 15%|█▌        | 2/13 [00:23<02:05, 11.38s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabri

Iteration 3/5


  0%|          | 0/13 [00:00<?, ?it/s]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
  8%|▊         | 1/13 [00:12<02:32, 12.68s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/wpy/miniconda3/envs/cellcharter/lib/python3.10 ...
  rank_zero_warn(
 15%|█▌        | 2/13 [00:27<02:34, 14.00s/it]/home/wpy/miniconda3/envs/cellcharter/lib/python3.10/site-packages/lightning_fabri

In [ ]:
spearmanr()